In [33]:
import sympy as sym
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import fsolve
from scipy.optimize import minimize
%matplotlib inline
from scipy.integrate import odeint, solve_ivp
import pandas as pd
import numpy as np

N_pop=331002647

C=pd.read_excel(r".\contact\usaM.xlsx")
D=C.to_numpy()
C=pd.read_excel(r".\contact\agdist.xlsx")
N=C.to_numpy()
N_total=N_pop*N

D=np.array([[1,1],[3,3]]).reshape(2,2)
N=np.array([0.6,0.4]).reshape(2,1)

gamma=1.

epsilon=0.1
k=len(N)
v=0.3
i0=np.zeros((k,1))+0.0001
v0=np.zeros((k,1))+v/k
nt=600
t_span=np.array([0,10])
times=np.linspace(t_span[0],t_span[1],2000)
FIG=0


def deriv( t,y,  beta, gamma,epsilon,D):
    #print(beta)
    S=(np.array(y[0:k])).reshape(k,1)
    V=(np.array(y[k:2*k])).reshape(k,1)
    I=(np.array(y[2*k:3*k])).reshape(k,1)
    dSdt = -beta*S* D.dot(I) 
    dVdt = -epsilon*V*beta*D.dot(I)
    dIdt = beta*(S+epsilon*V)*D.dot(I) - gamma * I
    x=np.vstack((dSdt,dVdt,dIdt))
    x=[item for sublist in x for item in sublist]
    return x
def func(v,beta, gamma,epsilon,D):
    v0=np.array(v).reshape(k,1)
    s0=N-i0-v0
    y0 =np.vstack((s0,v0,i0))
    y0 = [item for sublist in y0 for item in sublist]
    ret = solve_ivp(deriv, t_span, y0,t_eval=times,args=[beta,gamma,epsilon,D],rtol = 1e-9, atol = 1e-9)   
    #print(ret.y.shape)
    #Imax=max((sum(ret.y.T[2*k:3*k,:])).reshape(nt,1))
    ind=np.argmax(sum(ret.y[2*k:3*k,:]))
    S=(ret.y[0:k,ind]).reshape(k,1)
    V=ret.y[k:2*k,ind].reshape(k,1)
    I=ret.y[2*k:3*k,ind].reshape(k,1)
    loss=gamma/beta-(D[:,1].reshape(1,2)).dot(S+epsilon*V)
    print(loss)
    if FIG:
       fig = plt.figure(facecolor='w')
       ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
       ax.plot(times, sum(ret.y[2*k:3*k,:]), 'r', alpha=0.9, lw=2, label='Infected')
       plt.scatter(times[ind], sum(I),color='b')
    x=max(sum(ret.y[2*k:3*k,:]))
    #print(Imax)
    return x
#ret.y
def constraint1(x):
    return sum(x)-v
bnds=[]
for i in range(k):
   bnds.append(tuple([0,v-i0[i]]))

con1={'type':'eq','fun':constraint1}
cons=[con1]

#x=np.arange(8,9)

    #print(beta)

#A=np.array(B).reshape(k,len(x))
#for j in range(k):
#    fig = plt.figure(facecolor='w')
#    ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
#    ax.plot(x, A[j,:], 'r', alpha=0.9, lw=2, label='Infected')


    

    


In [32]:
D[:,1].reshape(2,1)


array([[1],
       [3]])

In [34]:
x=[6.]
B=[]
for beta in x:
    sol=minimize(func,v0,args=(beta,gamma,epsilon,D),method='SLSQP',bounds=bnds,constraints=cons,tol=1e-9)
    B.append(sol.x)
B

[[0.0008446]]
[[0.0008446]]
[[0.00084458]]
[[-0.00057283]]
[[-0.00057283]]
[[-0.00057285]]
[[-0.00011748]]
[[-0.00082775]]
[[-0.00082776]]
[[-0.00082777]]
[[-0.0006588]]
[[0.00065419]]
[[-0.00068166]]
[[0.00028674]]
[[0.0007932]]
[[0.00093525]]
[[0.00093524]]
[[0.00093523]]
[[-0.00088494]]
[[-0.00093089]]
[[-0.00094222]]
[[-0.00094223]]
[[-0.00094225]]
[[0.0009378]]
[[0.00093779]]
[[0.00093778]]
[[-0.00094357]]
[[-0.00094358]]
[[-0.0009436]]
[[0.00093844]]
[[0.00093882]]
[[0.00093881]]
[[0.0009388]]


[array([0.11294582, 0.18705418])]

In [430]:
beta=9.3
sol=minimize(func,v0,args=(beta,gamma,epsilon,D),method='SLSQP',bounds=bnds,constraints=cons,tol=1e-9)
func(sol.x,beta, gamma,epsilon,D)-func([0.,v],beta, gamma,epsilon,D)
#func([,v],beta, gamma,epsilon,D)


-0.032845900607062495

In [455]:
vt=B[0].reshape(2,1)
s0=N-i0-vt
gamma/(s0[0]+epsilon*vt[0]+2*(s0[1]+epsilon*vt[1]))

array([0.88526128])